In [19]:
import pandas as pd
import numpy as np
import torch

In [44]:
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.data import DataLoader


In [58]:
property_file = pd.read_csv('geometric_graph_200_property.txt', sep = '\t')
edge_idx_file = pd.read_csv('geometric_graph_200_edge_idx.txt', sep = ',',header = None)

x = torch.tensor(np.array(property_file), dtype=torch.float)
edge_idx = torch.tensor(np.array(edge_idx_file), dtype=torch.long)
y = torch.tensor([[1] for i in range(200)])
#print(edge_idx)
data = Data(x=x, edge_index=edge_idx.t().contiguous(), y =y)


In [72]:
num_nodes = 200
num_train_nodes = 120
num_valid_nodes = 30
num_test_nodes = 30
perm = torch.randperm(num_nodes)
train_idx = perm[:num_train_nodes]
valid_idx = perm[num_train_nodes:(num_train_nodes+num_valid_nodes)]
test_dix = perm[(num_train_nodes+num_valid_nodes):(num_train_nodes+num_valid_nodes+num_test_nodes)]

## debugging
#print(train_idx)
#print(valid_idx)

tensor([ 55, 158, 132, 126, 124,  87, 163,  98,  76, 152, 192, 169, 138,   7,
        150,  83, 191, 121, 140,  62,  47,  41,  13, 196, 120,   8,  40,  46,
        123, 171, 103, 195, 193, 145,  89, 165, 114, 184,  38,  52,  75, 129,
        182,  88,  67, 143,  50,  77, 136, 110,  66,  16, 173,  70, 137,  42,
         11, 162, 194,  71, 172,  45,  14,  44, 175, 100, 128, 167,  28, 160,
         35,  26, 161, 104,  48, 134, 108, 154, 112,  18, 149, 111, 148,  74,
        186,  59, 125,  43, 166, 180, 142, 159,  19, 127,  86,  73, 178,  49,
        106,  21,  22,  27, 117,  34,   1, 118, 156,   6,  93, 176,  17,  51,
         80,   4,  72,  57,  64, 179,  68,  30])
tensor([188,  12, 113,  33,  96,  65,  25, 197,  32, 198, 139,  15,  56,  91,
          3,  97, 181, 155,   5, 115, 116, 144, 168, 190, 122,  99, 135,   2,
         36,  94])


In [80]:
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(5, 256)
        self.conv2 = GCNConv(256,64)
        
        self.linear1 = nn.Linear(64,16)
        self.linear2 = nn.Linear(16, 6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)

        return F.log_softmax(x, dim=1)

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)



def train(data, train_idx):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[train_idx], data.y[train_idx])
    print(loss)
    loss.backward()
    optimizer.step()
    

In [82]:
for epoch in range(200):
    train(data,train_idx)

RuntimeError: multi-target not supported at /opt/conda/conda-bld/pytorch_1607369981906/work/aten/src/THCUNN/generic/ClassNLLCriterion.cu:15

In [ ]:

import os.path as osp
path = osp.join('/home/jiaqing/桌面/Fea2Fea/data/')

def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model(data)[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(data), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs
#------------------ Start our algorithm1 ---------------------#
if __name__ == '__main__':
    xlabels = ['Constant','Degree','Clustering_Coefficient','PageRank','Aver_Path_Len']
    for dataset,embedding_method in list(itertools.product(['Cora','PubMed','Citeseer'],['SAGE','GAT','GCN','GIN'])):
        
        Aver = np.zeros((5,5))
        dataset_name = dataset
        dataset = Planetoid(path, name = dataset, transform=T.NormalizeFeatures())
        data = dataset[0]

        name = r'/home/jiaqing/桌面/Fea2Fea/Result/Planetoid/' + dataset_name + '_property.txt'
        property_file = pd.read_csv(name, sep = '\t')
        
        

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        best_val_acc = test_acc = 0
        t = 0
        record_acc = 1
        total_epoch = 0 # for drawing curves
        avg_num = 10 # for iteration to count average accuracy
        
        for avg in range(avg_num):
            R = np.zeros((5,5)) # initialize our feature relationship matrix 
            R[0][0] = 1.000
            for i in range(2, 5):
                propert_i = property_file.iloc[:,[i]]
                array = np.array(propert_i)
                data.x = torch.tensor(array).float()
                for j in range(1,5):
                    propert_j = property_file.iloc[:,[j]]
                    array_2 = np.array(propert_j)
                    number = len(data.y)
                    data.y = binning(array_2, k = 6,data_len =  number)
                    
                    model =  Net(embedding=embedding_method).to(device) if embedding_method != 'MLP' else debug_MLP().to(device)
                    optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=1e-4)
                    #scheduler = StepLR(optimizer, step_size=10, gamma=0.8)
                    data =  data.to(device)
                    t = 0
                    best_val_acc = test_acc = 0 
                    for epoch in range(1, 2000):
                        
                        train()
                        train_acc, val_acc, tmp_test_acc = test()

                        if val_acc > best_val_acc:
                            best_val_acc = val_acc
                            test_acc = tmp_test_acc
                            if i == 0:
                                R[i][j] = round(test_acc,3)
                                R[j][i] = round(test_acc,3)
                            else:
                                R[i][j] = round(test_acc,3)
                            t = 0
                        t = t + 1
                        if t > 500:
                            break   
                        
                        log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
                        print(log.format(epoch, train_acc, best_val_acc, test_acc))
                        # do that for several times
                        #scheduler.step()
                    if i == 4 and j == 4:
                        Aver = Aver + R
                        